## 추천시스템 개발 코드

추천시스템은 고객별 최근 6개월의 데이터를 바탕으로 구축하였으며 6~3개월 전의 데이터를 1반기, 3개월~최근까지의 데이터를 2반기로 설정  
추천시스템은 2가지의 방식으로 구성되어 있으며 1번째 방식은 1반기에 많이 구매한 품목을 2반기에 구매하지 않은 경우 재구매를 추천하는것과  
2반기의 데이터를 바탕으로 knn최근접이웃방식과 잠재요인협업필터링의 방식을 통해 사용자,아이템사이의 유사도를 바탕으로 추천하는 방식 2가지를 사용  
이 중 2번째 방식에는 각 군집별로 가중치를 다르게 두어 맞춤형 추천시스템을 구현하였다.

### 1. 데이터 로딩 및 전처리

In [2]:
import pandas as pd
data = pd.read_csv('../../lmembers/data/purprod2.csv',encoding='ms949')
label_df = pd.read_excel('군집3(target_label=1).xlsx')
cate = pd.read_csv('../../lmembers/data/prodcl2.csv')
df = pd.merge(label_df,data)

In [3]:

def load_pdf(df,cate,N):
    data3 = df.loc[(df.구매일자>=20151001)&(df.라벨 == N)]
    data6 = df.loc[(df.구매일자<20151001)&(df.구매일자>=20150601)&(df.라벨 == N)]
    data3_cate = pd.merge(data3,cate,on='소분류코드')
    data6_cate = pd.merge(data6,cate,on='소분류코드')
    # 상품분류명별 구매횟수를 기준으로 아이템 추천하기 위하여 피벗 생성
    pdf3_ori = pd.pivot_table(data3_cate,
                        index = ['고객번호'],              # 행위치에 들어갈 열
                        columns = ['상품분류명'],         # 열위치에 들어갈 열
                        values = ['영수증번호'],              # 데이터로 사용할 열
                        aggfunc = ['count'])         # 데이터 집계 함수
    pdf6_ori = pd.pivot_table(data6_cate,
                        index = ['고객번호'],           
                        columns = ['상품분류명'],       
                        values = ['영수증번호'],           
                        aggfunc = ['count'])    
    pdf3=pdf3_ori.copy()
    pdf6=pdf6_ori.copy()
    # Nan값 0으로 대체 및 데이터프레임 정리
    pdf3.fillna(0,inplace=True)
    pdf3 = pdf3['count']['영수증번호']
    pdf6.fillna(0,inplace=True)
    pdf6 = pdf6['count']['영수증번호']
    return pdf3, pdf6

In [4]:
pdf3_0, pdf6_0 = load_pdf(df,cate,0)
pdf3_1, pdf6_1 = load_pdf(df,cate,1)
pdf3_2, pdf6_2 = load_pdf(df,cate,2)

### 2. 추천시스템 코드

**2.1 아이템 기반 최근접 이웃 협업 필터링**

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

def get_not_buy_list(ratings_matrix, custid):
    # custid로 입력받은 사용자의 모든 구매정보 추출하여 Series로 반환함. 

    # 반환된 user_rating 은 중분류명을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[custid,:]
    # user_rating이 0보다 크면 기존에 구매한 물품임. 대상 index를 추출하여 list 객체로 만듬
    already_buy = user_rating[ user_rating > 0].index.tolist()
    # 모든 중분류명을 list 객체로 만듬. 
    buy_list = ratings_matrix.columns.tolist()
    # 간편한 for문으로 already_buy에 해당하는 물품은 buy_list에서 제외함. 
    notbuy_list = [ item for item in buy_list if item not in already_buy]
    return notbuy_list

def recomm_item_by_custid(pred_df, custid, notbuy_list, top_n=10):
    # 구매예측 DataFrame에서 사용자id index와 notbuy_list로 들어온 중분류명 컬럼을 추출하여 가장 예측 구매횟수가 높은 순으로 정렬함. 
    recomm_items = pred_df.loc[custid, notbuy_list].sort_values(ascending=False)[:top_n]
    return recomm_items
    
def knn_recommend_pre(pdf3):
    # 코사인유사도는 행기준으로 작동하기에 transpose 함수로 전치
    pdf_T = pdf3.transpose()
    # 코사인 유사도 산출 (행인 아이템들을 기준으로 사용자간의 유사도)
    pdf_sim = cosine_similarity(pdf_T, pdf_T)
    # cosine_similarity()로 반환된 Numpy 행렬을 상품분류명으로 매핑해 DataFrame으로 변환
    pdf_sim_df = pd.DataFrame(data=pdf_sim, index=pdf3.columns,
                            columns=pdf3.columns)
    ratings_pred_knn = predict_rating(pdf3.values , pdf_sim_df.values)
    ratings_pred_matrix_knn = pd.DataFrame(data=ratings_pred_knn, index= pdf3.index,
                                    columns = pdf3.columns)
    
    return ratings_pred_matrix_knn

**2.2 잠재요인 협업 필터링**

In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 NULL이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

# 행렬 분해
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
    
    break_count = 0
    
    # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):  # steps는 SGD의 반복횟수
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:],Q[j,:].T)
            
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q, non_zeros)
        if ( step % 10) == 0: # 10회 반복할 때마다 오류 값 출력
            print(f'iteration step: {step}, rmse: {rmse}')
    return P, Q
    
def SGD_fac_recommend_pre(pdf3,steps=200,K=50,learning_rate=0.001,r_lambda = 0.001):
    P, Q = matrix_factorization(pdf3.values, steps=steps, K=K,learning_rate= learning_rate, r_lambda = r_lambda)
    pred_matrix = np.dot(P, Q.T)
    ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= pdf3.index,
                                   columns = pdf3.columns)
    return ratings_pred_matrix

In [7]:
# 1반기에 많이 구입한 물품 중 2반기 구입하지 않은 물품을 추천
def recom_notbuy(custid,pdf3,pdf6):
    user_rating = pdf6.loc[custid,:]
    already_buy = user_rating[ user_rating > 0].sort_values(ascending=False)
    notbuy_list = get_not_buy_list(pdf3, custid)
    for x in already_buy.index:
        if x in notbuy_list:
            already_buy.drop(index=x ,axis=0,inplace=True)
    recom_list = already_buy
    return recom_list

In [8]:
# sgd_0 = SGD_fac_recommend_pre(pdf3_0)
knn_0 = knn_recommend_pre(pdf3_0)

In [9]:
# sgd_1 = SGD_fac_recommend_pre(pdf3_1)
knn_1 = knn_recommend_pre(pdf3_1)

In [10]:
# sgd_2 = SGD_fac_recommend_pre(pdf3_2)
knn_2 = knn_recommend_pre(pdf3_2)

In [11]:
# sgd_0.to_csv('sgd_0')
# sgd_1.to_csv('sgd_1')
# sgd_2.to_csv('sgd_2')

In [12]:
sgd_0 = pd.read_csv('sgd_0',index_col=0)
sgd_1 = pd.read_csv('sgd_1',index_col=0)
sgd_2 = pd.read_csv('sgd_2',index_col=0)

### 3. 각 감소고객군집별로 가중치 부여

In [13]:
cate = pd.read_csv('../../lmembers/data/prodcl2.csv')
data_cate = pd.merge(data,cate,on='소분류코드')

In [14]:
list_A = data_cate.상품분류명[data_cate.제휴사_x == 'A'].unique().tolist()
list_B = data_cate.상품분류명[data_cate.제휴사_x == 'B'].unique().tolist()
list_C = data_cate.상품분류명[data_cate.제휴사_x == 'C'].unique().tolist()
list_편의 = data_cate.상품분류명[data_cate.소비재분류_x == '편의품'].unique().tolist()
list_선매 = data_cate.상품분류명[data_cate.소비재분류_x == '선매품'].unique().tolist()
list_전문 = data_cate.상품분류명[data_cate.소비재분류_x == '전문품'].unique().tolist()
list_레저 = data_cate.상품분류명[data_cate.통합분류_x == '레저/취미'].unique().tolist()
list_패션 = data_cate.상품분류명[data_cate.통합분류_x == '패션의류/잡화'].unique().tolist()
list_명품 = data_cate.상품분류명[data_cate.통합분류_x == '명품/쥬얼리'].unique().tolist()
list_식품 = data_cate.상품분류명[data_cate.통합분류_x == '식료품'].unique().tolist()
list_기호 = data_cate.상품분류명[data_cate.통합분류_x == '기호품'].unique().tolist()
list_생활 = data_cate.상품분류명[data_cate.통합분류_x == '생활잡화'].unique().tolist()

## 1번군집 통합 추천 시스템

In [15]:
# 1번군집 통합 추천 시스템
# 1번군집은 A사의 고급제품을 중심으로한 마케팅이 필요하기 때문에 A사와 전문품, 선매품과 매출이 많이 감소한 패션,명품, 레저에 가중치를 주었다.
def recommend_list_1(custid,pdf3,pdf6,knn,sgd,top = 50,N=5):
    notbuy_list = get_not_buy_list(pdf3, custid)
    for i in knn.columns:
        if i in list_A:
            knn[i] = knn[i] *1.1
        if i in list_전문:
            knn[i] = knn[i] *1.1
        if i in list_선매:
            knn[i] = knn[i] *1.1
        if i in list_레저:
            knn[i] = knn[i] *1.1
        if i in list_패션:
            knn[i] = knn[i] *1.1
        if i in list_명품:
            knn[i] = knn[i] *1.1
    recomm_items_knn = recomm_item_by_custid(knn, custid, notbuy_list, top_n=top)
    recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
    recomm_items_SGD = recomm_item_by_custid(sgd, custid, notbuy_list, top_n=top)
    recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
    recomm_items_SGD.index.name='상품분류명'
    recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='상품분류명')
    recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
    recom_items = recom_items.sort_values(by='recom_pred',ascending=False)
    recom_list = recom_notbuy(custid,pdf3,pdf6)
    print(f'3개월전에 {recom_list.index.values[:N]}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_items.index.values[:N]}은/는 어떠세요?')

In [1]:
cust_0 = np.random.choice(df.고객번호[df.라벨 == 0].unique().tolist())
recommend_list_1(cust_0,pdf3_0,pdf6_0,knn_0,sgd_0)

NameError: name 'np' is not defined

## 2번군집 통합 추천 시스템

In [33]:
# 2번군집 통합 추천 시스템
# 2번군집은 A,B사의 편의품분류와 식료품, 기호품을 중심으로 마케팅이 필요하기에 이에 가중치를 부여
def recommend_list_2(custid,pdf3,pdf6,knn,sgd,top = 50,N=5):
    notbuy_list = get_not_buy_list(pdf3, custid)
    for i in knn.columns:
        if i in list_A:
            knn[i] = knn[i] *1.1
        if i in list_B:
            knn[i] = knn[i] *1.1
        if i in list_편의:
            knn[i] = knn[i] *1.1
        if i in list_식품:
            knn[i] = knn[i] *1.1
        if i in list_기호:
            knn[i] = knn[i] *1.1
        if i in list_선매:
            knn[i] = knn[i] *1.05  
        if i in list_패션:
            knn[i] = knn[i] *1.05  
        
    recomm_items_knn = recomm_item_by_custid(knn, custid, notbuy_list, top_n=top)
    recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
    recomm_items_SGD = recomm_item_by_custid(sgd, custid, notbuy_list, top_n=top)
    recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
    recomm_items_SGD.index.name='상품분류명'
    recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='상품분류명')
    recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
    recom_items = recom_items.sort_values(by='recom_pred',ascending=False)
    recom_list = recom_notbuy(custid,pdf3,pdf6)
    print(f'3개월전에 {recom_list.index.values[:N]}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_items.index.values[:N]}은/는 어떠세요?')

In [46]:
cust_1 = np.random.choice(df.고객번호[df.라벨 == 1].unique().tolist())
recommend_list_2(cust_1,pdf3_1,pdf6_1,knn_1,sgd_1)

3개월전에 ['커피음료' '차/커피' '냉장음료' '라면' '담배']물품들을 구매하셨는데 지금은 필요하지 않으신가요?

13682고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 ['생수' '전통주' '스포츠의류' '가공식품' '농산물']은/는 어떠세요?


## 3번군집 통합 추천 시스템

In [35]:
# 3번군집 통합 추천 시스템
# 3번군집은 1,2번군집보다 구매가 고르게 분산되어 있으므로 가중치도 분산되게 부여 
# C사의 비중이 군집중 가장높고 편의,전문품 분류의 감소와 기호품,식료품,명품,생활잡화의 감소가 높으므로 가중치를 부여
def recommend_list_3(custid,pdf3,pdf6,knn,sgd,top = 50,N=5):
    notbuy_list = get_not_buy_list(pdf3, custid)
    for i in knn.columns:
        if i in list_C:
            knn[i] = knn[i] *1.1
        if i in list_편의:
            knn[i] = knn[i] *1.05
        if i in list_전문:
            knn[i] = knn[i] *1.05
        if i in list_기호:
            knn[i] = knn[i] *1.05
        if i in list_식품:
            knn[i] = knn[i] *1.05
        if i in list_명품:
            knn[i] = knn[i] *1.05
        if i in list_생활:
            knn[i] = knn[i] *1.05
    recomm_items_knn = recomm_item_by_custid(knn, custid, notbuy_list, top_n=top)
    recomm_items_knn = pd.DataFrame(data=recomm_items_knn.values,index=recomm_items_knn.index,columns=['pred_score'])
    recomm_items_SGD = recomm_item_by_custid(sgd, custid, notbuy_list, top_n=top)
    recomm_items_SGD = pd.DataFrame(data=recomm_items_SGD.values,index=recomm_items_SGD.index,columns=['pred_score'])
    recomm_items_SGD.index.name='상품분류명'
    recom_items = pd.merge(recomm_items_SGD,recomm_items_knn,on='상품분류명')
    recom_items['recom_pred'] = recom_items.iloc[:,1]*recom_items.iloc[:,0]
    recom_items = recom_items.sort_values(by='recom_pred',ascending=False)
    recom_list = recom_notbuy(custid,pdf3,pdf6)
    print(f'3개월전에 {recom_list.index.values[:N]} 물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_items.index.values[:N]}은/는 어떠세요?')

In [47]:
cust_2 = np.random.choice(df.고객번호[df.라벨 == 2].unique().tolist())
recommend_list_3(cust_2,pdf3_2,pdf6_2,knn_2,sgd_2)

3개월전에 ['대형가전' '통신/컴퓨터' '발효유' '과채과일' '우유'] 물품들을 구매하셨는데 지금은 필요하지 않으신가요?

97고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 ['맥주' '커피음료' '소주' '생활잡화' '식사류']은/는 어떠세요?
